# Lab 4 - Model Deplyoment 

In this lab, you will learn how to use Azure Machine Learning Service to deploy, manage, and monitor the trained models.


The following diagram illustrates the complete deployment workflow.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/model-ci-cd.png)



The deployment workflow includes the following steps:

- Create/Retrain the model
- Register the model in a registry hosted in your Azure Machine Learning Service workspace
- Register an image that pairs a model with a scoring script and dependencies in a portable container
- Deploy the image as a web service in the cloud or to edge devices
- Monitor and collect data


You completed the first two steps in the previous labs.

In this lab we will walk-through the reminder of the deployment workflow.



## Connect to the workspace

In [2]:
# Verify AML SDK Installed
# view version history at https://pypi.org/project/azureml-sdk/#history 
import azureml.core
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.0.2


In [3]:
from azureml.core import Workspace

# Read the workspace config from file
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Found the config file in: /data/home/demouser/notebooks/MTC_AzureAILabs/ML-AML-Walkthrough/aml_config/config.json
jkamlworkshop
jkamlworkshop
southcentralus
952a710c-8d9c-40c1-9fec-f752138cc0b3


## Retrieve the model from AML Model Registry.

Download the model registered in the previous step.


In [32]:
from azureml.core.model import Model

# Download the model to a local directory
model_name = 'propensity_to_buy_predictor'
model_path = Model.get_model_path(model_name, _workspace=ws)

# Re-load the model
model = pickle.load(open(os.path.join(model_path, model_name + '.pkl'), 'rb'))
print("Loaded model from:", os.path.join(model_path, model_name + '.pkl'))

# Re-load the scaler
scaler = pickle.load(open(os.path.join(model_path, scaler_name + '.pkl'),'rb'))
print("Loaded scaler from:", os.path.join(model_path, scaler_name + '.pkl'))

# Run a quick test
age = 60
km = 40000

scaled_input = scaler.transform([[age, km]])
prediction = model.predict(scaled_input)
print(prediction)

Model name: AutoML5178497cbbest


## Create and deploy the container image encapsulating the model

When you deploy a model using AML to either ACI or AKS, you are deploying a Docker container encapsulating a trained model, its dependencies, and a web services wrapper around the model. 

### Create scoring script
Create the scoring script, called score.py, used by the web service call to invoke the model.

You must include two required functions in the scoring script:

- The `init()` function, which loads the model into a global object. This function is run only once when the Docker container is started.

- The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats can be used.

In [4]:
%%writefile score.py

import json
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import pickle
from sklearn.externals import joblib

def init():
    try:
        # One-time initialization of predictive model and scaler
        from azureml.core.model import Model
        
        global model   

        model_name = '<<modelid>>'
        model_path = Model.get_model_path(model_name, _workspace=ws)
        model = pickle.load(open(model_path, 'rb'))
        
    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):     
    try:
        inputs = json.loads(input_json)

        #Get the scored result
        prediction = json.dumps(model.predict(inputs).tolist())

    except Exception as e:
        prediction = str(e)
    return prediction


Overwriting score.py


Substitute the actual model ID in the script file.

In [5]:
from azureml.core.model import Model

model_name = 'propensity_to_buy_predictor'
model = Model(ws, name=model_name)
script_file_name = 'score.py'

with open(script_file_name, 'r') as cefr:
    content = cefr.read()
    
with open(script_file_name, 'w') as cefw:
    cefw.write(content.replace('<<modelid>>', model.name))

Review the updated script.

In [6]:
with open("score.py","r") as f:
    print(f.read())


import json
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import pickle
from sklearn.externals import joblib

def init():
    try:
        # One-time initialization of predictive model and scaler
        from azureml.core.model import Model
        
        global model   

        model_name = 'propensity_to_buy_predictor'
        model_path = Model.get_model_path(model_name, _workspace=ws)
        model = pickle.load(open(model_path, 'rb'))
        
    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):     
    try:
        inputs = json.loads(input_json)

        #Get the scored res

------------ Sandbox ---------------

In [12]:
import json
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import pickle
from sklearn.externals import joblib

def init():
    try:
        # One-time initialization of predictive model and scaler
        from azureml.core.model import Model
        
        global model   

        model_name = 'propensity_to_buy_predictor'
        model_path = Model.get_model_path(model_name, _workspace=ws)
        #model = pickle.load(open(model_path, 'rb'))
        print(model_path)
        
    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):     
    try:
        inputs = json.loads(input_json)

        #Get the scored result
        prediction = json.dumps(model.predict(inputs).tolist())

    except Exception as e:
        prediction = str(e)
    return prediction


In [13]:
init()

azureml-models/propensity_to_buy_predictor/6/model.pkl


In [14]:
model_name = 'propensity_to_buy_predictor'
model_path = Model.get_model_path(model_name, _workspace=ws)

In [17]:
from sklearn.externals import joblib

model = joblib.load(model_path)

/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


------------- Sandbox --------------

### Create a Conda dependencies environment file.

Next, create an environment file that specifies the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. 

To ensure the consistency of the prediction results with the training results, the AML SDK dependency versions used by the scoring environment needs to be the same as in the environment that was used to train the model.

The SDK dependency versions used to train the model can be retrieved from the run history.

You need to replace the values in `experiment_name`  with the name of your experiment.


In [35]:
from azureml.core import Experiment
from azureml.train.automl.run import AutoMLRun

# Retrieve training run record
experiment_name = 'propensity_to_buy_classifier_automatedml'
run_id = model.tags['RunID'] 

experiment = Experiment(ws, experiment_name)
automl_run = AutoMLRun(experiment = experiment, run_id = run_id)

# Retrieve SDK dependencies. 
dependencies = automl_run.get_run_sdk_dependencies(iteration = 0, check_versions=False)
print(dependencies)

{'azureml-train-automl': '1.0.6', 'azureml-telemetry': '1.0.6', 'azureml-explain-model': '1.0.6', 'azureml-defaults': '1.0.6', 'azureml-dataprep': '1.0.6', 'azureml-dataprep-native': '11.2.0', 'azureml-core': '1.0.6'}


Create conda environment file.

In [56]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
#myenv.add_channel('defaults')
#myenv.add_pip_package("azureml-sdk[notebooks,automl]==" + sdk_version)
#myenv.remove_pip_package('azureml-defaults')
for package, version in dependencies.items():
    myenv.add_pip_package('{}=={}'.format(package, version))
    
with open("mydeployenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [7]:
from azureml.core.conda_dependencies import CondaDependencies 

#mycondaenv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'])
mycondaenv = CondaDependencies.create()

with open("mydeployenv.yml","w") as f:
    f.write(mycondaenv.serialize_to_string())

Review the content of 'yml' file.

In [8]:
with open("mydeployenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults==1.0.2



### Create docker image for deployment

To create a Container Image, you need four things: the model metadata (as retrieved from Model Registry), the scoring script file, the runtime configuration (defining whether Python or PySpark should be used) and the Conda Dependencies file.

In [9]:
from azureml.core.image import ContainerImage, Image

# Define runtime
runtime = "python" 

# Define scoring script
driver_file = "score.py"

# Define conda dependencies
conda_file = "mydeployenv.yml"

# configure the image
image_config = ContainerImage.image_configuration(execution_script=driver_file, 
                                                  runtime=runtime, 
                                                  conda_file=conda_file,
                                                  description="Image for propensity to buy predictor",
                                                  tags={"Classifier": "AutomatedML"})

image = Image.create(name = "propensity-to-buy-classifier",
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)


Creating image
Running...................
SucceededImage creation operation finished for image propensity-to-buy-classifier:4, operation "Succeeded"


### Deploy the container image to ACI

With the Container Image  in hand, you are almost ready to deploy to ACI. The next step is to define the size of the VM that ACI will use to run your Container.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Azure ML ACI'}, 
    description = 'This is a deployment of the propensity to buy predictor.')

At this point you can deploy the image to the webservice to ACI

In [ ]:
from azureml.core.webservice import Webservice

service_name = "usedcarsmlservice-aci"
print("Deploying: ", service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aci_config,
                                           image = image,
                                           name = service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)

### Test the service

Once the webservice deployment completes, you can use the returned webservice object to invoke the webservice. 

In [ ]:
import json
age = 60
km = 40000
test_data  = json.dumps([[age,km]])
test_data
result = aci_service.run(input_data=test_data)
print(result)

### Deploy the container image to AKS

Once you are familiar with the process for deploying a webservice to ACI, you will find the process for deploying to AKS to be similar with one additional step that creates the AKS cluster first.

In [ ]:
# Provision an AKS cluster 

from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Use the default configuration, overriding the default location to a known region that supports AKS
prov_config = AksCompute.provisioning_configuration(location='westus2')

aks_name = 'aks-cluster01' 

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)


# Wait for cluster to be ready
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

With your AKS cluster ready, now you can deploy your webservice. Once again, you need to provide a configuration for the size of resources allocated from the AKS cluster to run instances of your Container.

In [ ]:
# Create the web service configuration (using defaults)
aks_config = AksWebservice.deploy_configuration()

aks_service_name ='usedcarsmlservice-aks'

aks_service = Webservice.deploy_from_image(
  workspace=ws, 
  name=aks_service_name, 
  image = image,
  deployment_target=aks_target
  )


aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

### Test the service
As before, you can use the webservice object returned by the deploy_from_model method to invoke your deployed webservice. 

In [ ]:
import json
age = 60
km = 40000
test_data  = json.dumps([[age,km]])
test_data
result = aks_service.run(input_data=test_data)
print(result)

## Clean up

Make sure to remove ACI and AKS deployments. Use Azure Portal to remove *Deployments* and *AKS Compute*.